In [1]:
# %%capture
# Capture because braindecode module prints a lot :S
from pyFiles.training import *
from pyFiles.models import *
from pyFiles.dataprep_down import *
from pyFiles.dataprep2_down import *
#
import torch
from torch import nn
from torch.nn import functional as F
#
from pyFiles.losses import InfoNceLoss, VICReg, BarlowTwins

c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\docstring_inheritance\docstring_inheritors\bases\inheritor.py:329: DocstringInheritanceWarning: File c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\braindecode\models\eegconformer.py:88: in EEGConformer.__init__: section Parameters: the docstring for the argument 'n_outputs' is missing.
  self._warn(
c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\docstring_inheritance\docstring_inheritors\bases\inheritor.py:329: DocstringInheritanceWarning: File c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\braindecode\models\eegconformer.py:88: in EEGConformer.__init__: section Parameters: the docstring for the argument 'n_chans' is missing.
  self._warn(
c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\docstring_inheritance\docstring_inheritors\bases\inheritor.py:329: DocstringInheritanceWarning: File c:\Users\borja\miniconda3\envs\neuron\Lib\site-packages\braindecode\models\eegconformer.py:88: in EEGConformer.

In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [3]:
#PREP DENUEVO
# prep("C:\\Users\\TheSy\\Desktop\\dataverse_files", mode = "per_win", save = True, save_dir = "data_down")

In [4]:
# spliter = DFSpliter(save = True, mode= "down_win")
# spliter("down_prep_windows.csv",".")

In [5]:
transforms = Augmentations(5, False, AUGMENTATIONS)

In [6]:
p_size = 32
class CustomEEGDataset(Dataset):
    def __init__(self, csv_file , root_dir , transform = None, ):

        try:
            self.loc_df = pd.read_csv(os.path.join(root_dir,csv_file)).drop(labels="Unnamed: 0", axis = 1)
        except:
            self.loc_df = pd.read_csv(os.path.join(root_dir,csv_file))

        self.transform = transform
        self.root_dir = root_dir

    def __len__(self,):
        return len(self.loc_df)//p_size
    

    def __getitem__(self, idx):
        exam_shape = (p_size, 19, 1000)
        # print(val_shape)
            # return ch, diagnostic
        new_exam = torch.ones(exam_shape)
        # for i in range(len(self.loc_df)):
        for j in range(p_size):
            eeg_file = os.path.join(self.root_dir,
                            self.loc_df.iloc[idx*p_size + j, 2])
            ch, diagnostic = torch.load(eeg_file)
        
            if self.transform is not None:
                ch = self.transform(ch)
            new_exam[j] = ch
        
        return new_exam, diagnostic

In [7]:
train_dataset = CustomEEGDataset("down_win_train_feats.csv", root_dir = "", transform = transforms)
val_dataset = CustomEEGDataset("down_win_val_feats.csv", root_dir = "",)

In [8]:
# trainloader = DataLoader(train_dataset, batch_size = 1, shuffle = True, num_workers = 0)
trainloader = DataLoader(train_dataset, batch_size = 22, shuffle = True, num_workers = 2)
valloader = DataLoader(val_dataset, batch_size = 12, shuffle = False, num_workers = 2)

In [10]:
model = SleepStager(n_chans = 19, pretext = False, p_size = 32)
enc, fc = torch.load("D:\\bestEncoderParamsmulti_inf.pt")
# enc, fc = torch.load("bestEncoderParamsmulti_VR.pt")
# model.encoder.load_state_dict(enc)

criterion = nn.CrossEntropyLoss(label_smoothing= 0.1)
# criterion = nn.MSELoss()

#FREEZE
# for param in model.encoder.parameters():
#   param.requires_grad = False

optimizer = torch.optim.Adam(
    [
        {"params": model.encoder.parameters(), "lr": 1e-7},
        {"params": model.final.parameters(), "lr": 3e-4},
    ],
    lr=5e-4,
) 
downtrain(model, 30, trainloader, valloader, criterion, optimizer, state = None, name = "multi_inf")

Epoch nro 1/30
Iter: 1/2, Loss:0.691448450088501
Iter: 2/2, Loss:0.7117933034896851
Epoch 1/30: Train loss = 0.701620876789093, Val loss = 0.6904094815254211, Train acc = 0.5, Val acc = 0.5
Better params found in epoch = 1, saved params
Epoch nro 2/30
Iter: 1/2, Loss:0.7021655440330505
Iter: 2/2, Loss:0.6562069654464722
Epoch 2/30: Train loss = 0.6791862547397614, Val loss = 0.6686797738075256, Train acc = 0.5909090909090909, Val acc = 0.6666666666666666
Better params found in epoch = 2, saved params
Epoch nro 3/30
Iter: 1/2, Loss:0.7022897005081177
Iter: 2/2, Loss:0.6453142762184143
Epoch 3/30: Train loss = 0.673801988363266, Val loss = 0.646470308303833, Train acc = 0.5909090909090909, Val acc = 0.6666666666666666
Better params found in epoch = 3, saved params
Epoch nro 4/30
Iter: 1/2, Loss:0.6364813446998596
Iter: 2/2, Loss:0.6599710583686829
Epoch 4/30: Train loss = 0.6482262015342712, Val loss = 0.6302335858345032, Train acc = 0.6363636363636364, Val acc = 0.6666666666666666
Bette

([[0.701620876789093,
   0.6791862547397614,
   0.673801988363266,
   0.6482262015342712,
   0.6443010866641998,
   0.6568185091018677,
   0.6352861225605011,
   0.6293875575065613,
   0.6243624091148376,
   0.620988667011261,
   0.5890771448612213,
   0.6087709367275238,
   0.5817777812480927,
   0.5688978135585785,
   0.5518865585327148,
   0.5349545478820801,
   0.5767553150653839,
   0.5515175461769104,
   0.5583126246929169,
   0.5329703539609909,
   0.5040745735168457,
   0.49178749322891235,
   0.49361446499824524,
   0.4872267097234726,
   0.5046198666095734,
   0.490585595369339,
   0.4692552387714386,
   0.46618524193763733,
   0.448899507522583,
   0.40847088396549225],
  [0.6904094815254211,
   0.6686797738075256,
   0.646470308303833,
   0.6302335858345032,
   0.6289387941360474,
   0.6423031687736511,
   0.6193150877952576,
   0.5943642854690552,
   0.585701048374176,
   0.5830637216567993,
   0.5818653106689453,
   0.5804641842842102,
   0.5711920857429504,
   0.56810641

In [ ]:
model

SleepStager(
  (encoder): SleepStagerChambon2018(
    (spatial_conv): Conv2d(1, 19, kernel_size=(19, 1), stride=(1, 1))
    (feature_extractor): Sequential(
      (0): Conv2d(1, 16, kernel_size=(1, 50), stride=(1, 1), padding=(0, 25))
      (1): Identity()
      (2): ReLU()
      (3): MaxPool2d(kernel_size=(1, 13), stride=(1, 13), padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(16, 16, kernel_size=(1, 50), stride=(1, 1), padding=(0, 25))
      (5): Identity()
      (6): ReLU()
      (7): MaxPool2d(kernel_size=(1, 13), stride=(1, 13), padding=0, dilation=1, ceil_mode=False)
    )
  )
  (final): RNNClassifier(
    (rnn): RNN(1824, 100, batch_first=True)
    (fc): Linear(in_features=100, out_features=2, bias=True)
  )
)